# Code

In [33]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
from datetime import datetime
import time


In [34]:
username = 'amiramirmashgh'
password = '1234Asdf'
# username, password
tv = TvDatafeed()

In [35]:
def analize_data(df_XUD):
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # ADD diff and color off the candle

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')

    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy'
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] )
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)

    def determine_rule3(row):
        if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
            return "Half_buy"
        elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
            return "sell"
        else:
            return  'Half_sell'

    df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1)

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1)

    def determine_finall_desions(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy"):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy"):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1)

    def determine_finall_desions_4hour(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True"  and row["rule_4_5"] == "buy"):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True"  and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_4_5"] == "buy"):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions_4h'] = df_XUD.apply(determine_finall_desions_4hour, axis=1)

    # print(df_XUD)
    #last candle must delete?

    # Save the DataFrame to a CSV file
    # df_XUD.to_csv('output.csv', index=True)
    # df_XUD.drop(df_XUD.index[-1], inplace=False)
    df_XUD = df_XUD.drop(df_XUD.index[-1])

    # print("DataFrame saved to 'output.csv'")
    return df_XUD

In [36]:
def get_data_5_min(coin):
    try:
      # get data
      df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_5_minute,n_bars=300)
    except:
      print("error \n error " , coin)
    return df_XUD

def get_data_30_min(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_30_minute,n_bars=200)
    return df_XUD
def get_data_1_hour(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_1_hour,n_bars=200)
    return df_XUD
def get_data_4_hour(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_4_hour,n_bars=200)
    return df_XUD


def get_data_5_min_forex(coin):
    # get data
    try:
      df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_5_minute,n_bars=300)
    except:
      print("error \n error " , coin)

    return df_XUD

def get_data_30_min_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_30_minute,n_bars=200)
    return df_XUD
def get_data_1_hour_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_1_hour,n_bars=200)
    return df_XUD
def get_data_4_hour_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_4_hour,n_bars=200)
    return df_XUD

In [37]:
# caculate change_percent

import pandas as pd
from datetime import datetime

# Get today's date

def caculate_change_percent(price_open_of_the_new_day,live_price):
  return abs(price_open_of_the_new_day - live_price) / price_open_of_the_new_day *100

def change_percent(df_XUD):
  today_date = datetime.now().date()
  # Extract the date part (midnight) from today's date
  price_open_of_the_new_day = df_XUD[df_XUD.index.date == today_date].iloc[0]["open"]
  return caculate_change_percent(price_open_of_the_new_day ,df_XUD.iloc[-1]["close"] )



In [43]:
list_crypto =["GTCUSDT.P","KNCUSDT.P","HBARUSDT.P","BTCUSDT.P","MKRUSDT.P","LDOUSDT.P","KAVAUSDT.P","MATICUSDT.P","QNTUSDT.P", "FTMUSDT.P","ADAUSDT.P","BNBUSDT.P","ETHUSDT.P","APEUSDT.P","APTUSDT.P","PEOPLEUSDT.P","NEOUSDT.P","GRTUSDT.P","SNXUSDT.P","SOLUSDT.P","VETUSDT.P", "AXSUSDT.P", "XRPUSDT.P","XLMUSDT.P","ALGOUSDT.P","FILUSDT.P","EOSUSDT.P","AXSUSDT.P","ATOMUSDT.P","CHZUSDT.P","DOTUSDT.P","ARBUSDT.P","ETCUSDT.P","ETCUSDT.P","TRXUSDT.P","SANDUSDT.P","MANAUSDT.P","AAVEUSDT.P","EGLDUSDT.P","LTCUSDT.P","SNXUSDT.P","BCHUSDT.P","RNDRUSDT.P","UNIUSDT.P","AVAXUSDT.P","VETUSDT.P","THETAUSDT.P","XLMUSDT.P","KLAYUSDT.P","XMRUSDT.P","NEARUSDT.P","INJUSDT.P","HBARUSDT.P","ICPUSDT.P","IOTAUSDT.P","TUSDT.P","DASHUSDT.P","1INCHUSDT.P","ZECUSDT.P","TOMOUSDT.P","DUSKUSDT.P","JOEUSDT.P","STXUSDT.P","ANTUSDT.P","CFXUSDT.P","OPUSDT.P","HIGHUSDT.P","BLZUSDT.P","1000PEPEUSDT.P","KSMUSDT.P","SUSHIUSDT.P","STGUSDT.P","LEVERUSDT.P","CTKUSDT.P","DYDXUSDT.P","PHBUSDT.P","CELRUSDT.P","UNFIUSDT.P","1000SHIBUSDT.P","COMBOUSDT.P","IOTXUSDT.P","SKLUSDT.P","1000FLOKIUSDT.P","KNCUSDT.P","GTCUSDT.P","TRUUSDT.P","COTIUSDT.P","RDNTUSDT.P","ZRXUSDT.P","WAVESUSDT.P","1000XECUSDT.P","EDUUSDT.P","DOGEUSDT.P","AUDIOUSDT.P","MAGICUSDT.P","BLUEBIRDUSDT.P","MASKUSDT.P","KEYUSDT.P","RADUSDT.P","ACHUSDT.P","BALUSDT.P","SXPUSDT.P","SUIUSDT.P","WOOUSDT.P","1000LUNCUSDT.P","ENJUSDT.P","RVNUSDT.P","LUNA2USDT.P","SFPUSDT.P","ZILUSDT.P","ONTUSDT.P","CTSIUSDT.P","BLURUSDT.P","YFIUSDT.P","OMGUSDT.P","GALUSDT.P","DEFIUSDT.P","ALPHAUSDT.P","BELUSDT.P","HOOKUSDT.P","REEFUSDT.P","NKNUSDT.P","IMXUSDT.P","ROSEUSDT.P","HOTUSDT.P","MINAUSDT.P","MTLUSDT.P","ARPAUSDT.P","LINKUSDT.P","IOSTUSDT.P","FETUSDT.P","BATUSDT.P","XVSUSDT.P","IDUSDT.P","IDUSDT.P","AGIXUSDT.P","AMBUSDT.P","QTUMUSDT.P","LQTYUSDT.P","OCEANUSDT.P","BNXUSDT.P","SSVUSDT.P","LINAUSDT.P","CHRUSDT.P","RENUSDT.P","LPTUSDT.P","ARUSDT.P","TLMUSDT.P","JASMYUSDT.P","GMXUSDT.P","FXSUSDT.P","CKBUSDT.P","SPELLUSDT.P","ICXUSDT.P","GMTUSDT.P","ASTRUSDT.P","ENSUSDT.P","C98USDT.P","UMAUSDT.P","IDEXUSDT.P","STORJUSDT.P","GALAUSDT.P","BTCDOMUSDT.P","ANKRUSDT.P","HFTUSDT.P","FOOTBALLUSDT.P","COMPUSDT.P","ZENUSDT.P","RLCUSDT.P","BANDUSDT.P","LRCUSDT.P","RSRUSDT.P","PERPUSDT.P","CVXUSDT.P"]
list_forex = ["EURUSD","USDJPY","USDCHF","USDCAD","AUDUSD","GBPUSD","NZDUSD","EURNZD","EURCHF","EURGBP","EURCAD","EURAUD","EURJPY","GBPJPY","GBPNZD","GBPCHF","GBPCAD","GBPAUD","CADJPY","AUDJPY","CHFJPY","NZDJPY","AUDCHF","CADCHF","NZDCHF","AUDNZD","AUDCAD","NZDCAD"]
def extract_signal_5_min(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", "ch per = ", "{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0



def extract_signal_30_min(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0


def extract_signal_1_hour(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0

def extract_signal_4_hour(df_XUD,time):
    if df_XUD["finall_desions_4h"][-1] == "buy" or df_XUD["finall_desions_4h"][-1] == "buy_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    elif df_XUD["finall_desions_4h"][-1] == "sell" or df_XUD["finall_desions_4h"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", "ch per = ","{:.2f}".format(change_percent(df_XUD)),"] ",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0

In [45]:
import concurrent.futures
import multiprocessing

def analyze_crypto_5min(crypto):
    df_XUDD = analize_data(get_data_5_min(crypto))
    extract_signal_5_min(df_XUDD, datetime.now().minute)
def analyze_crypto_30min(crypto):
    df_XUDD = analize_data(get_data_30_min(crypto))
    extract_signal_30_min(df_XUDD, datetime.now().minute)
def analyze_crypto_1hour(crypto):
    df_XUDD = analize_data(get_data_1_hour(crypto))
    extract_signal_1_hour(df_XUDD, datetime.now().minute)
def analyze_crypto_4hour(crypto):
    df_XUDD = analize_data(get_data_4_hour(crypto))
    extract_signal_4_hour(df_XUDD, datetime.now().minute)

def analyze_pair_coin_5min(pair_coin):
    print(pair_coin)
    df_XUDD = analize_data(get_data_5_min_forex(pair_coin))
    extract_signal_5_min(df_XUDD, datetime.now().minute)

def analyze_pair_coin_30min(pair_coin):
    df_XUDD = analize_data(get_data_30_min_forex(pair_coin))
    extract_signal_30_min(df_XUDD, datetime.now().minute)

def analyze_pair_coin_1hour(pair_coin):
    df_XUDD = analize_data(get_data_1_hour_forex(pair_coin))
    extract_signal_1_hour(df_XUDD, datetime.now().minute)
def analyze_pair_coin_4hour(pair_coin):
    df_XUDD = analize_data(get_data_4_hour_forex(pair_coin))
    extract_signal_4_hour(df_XUDD, datetime.now().minute)


In [40]:

def extract_break_SMA20_4h(df_XUD,time):
    if df_XUD["inside candle (rule 1)"][-1] == "buy":
        print("time :" , time ,df_XUD["symbol"][-1] , "buy" , "4 hour time frame , looking 5 min to enter")
        return 1
    elif df_XUD["inside candle (rule 1)"][-1] == "sell":
        print("time :" , time ,df_XUD["symbol"][-1], "sell" , "4 hour time frame , looking 5 min to enter")
        return -1
    else :
        return 0


def extract_break_SMA20_5min_buy(df_XUD):
      if df_XUD["inside candle (rule 1)"][-1] == "buy":
        if df_XUD["SMA 20"][-2] - df_XUD["close"][-2] > 0 and df_XUD["cross_counter"][-1] < 4:
          print(df_XUD["symbol"][-1] , "buy" , "5 min entry founded,must enter")
          return 1
        elif df_XUD["SMA 20"][-2] - df_XUD["close"][-2] < 0:
          print(df_XUD["symbol"][-1] , "buy" , "5 min entry founded,must enter")
          return 1
        else:
          return 0
      else :
        return 0

def extract_break_SMA20_5min_sell(df_XUD):
      if df_XUD["inside candle (rule 1)"][-1] == "sell":
        if df_XUD["SMA 20"][-2] - df_XUD["close"][-2] < 0 and df_XUD["cross_counter"][-1] < 4:
          print(df_XUD["symbol"][-1] , "sell" , "5 min entry founded,must enter")
          return 1
        elif df_XUD["SMA 20"][-2] - df_XUD["close"][-2] > 0:
          print(df_XUD["symbol"][-1] , "sell" , "5 min entry founded,must enter")
          return 1
        else:
          return 0
      else :
        return 0


def analyze_pair_coin_4hour_5min(pair_coin):
    df_XUDD = analize_data(get_data_4_hour_forex(pair_coin))
    return extract_break_SMA20_4h(df_XUDD, datetime.now().minute)

def analyze_crypto_4hour_5min(crypto):
    df_XUDD = analize_data(get_data_4_hour(crypto))
    return extract_break_SMA20_4h(df_XUDD, datetime.now().minute)

def find_accepted_entry_5_min_crypto(coin,position_type):
    df_XUDD = analize_data(get_data_5_min(coin))
    if position_type == "buy":
      return extract_break_SMA20_5min_buy(df_XUDD)
    elif position_type == "sell":
      return extract_break_SMA20_5min_sell(df_XUDD)
    else:
      return 0

In [46]:
def main():
    counter_4hour = 0
    while(True):
      current_time = datetime.now()
      minutes_5 = int(str(current_time.minute))%5
      minutes_30 = int(str(current_time.minute))%30
      minutes_1_hour = int(str(current_time.minute))
      hour_4  = int(str(current_time.hour))%4
      second  = current_time.second
      # # 4 hour 7 rules
      # if (minutes_1_hour == 0) and second <= 1 and hour_4 == 0:
      #     print("\n",current_time.hour,":",current_time.minute , "For 4 hour we got :")
      #     with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
      #        executor.map(analyze_pair_coin_4hour, list_forex)
      #     with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
      #        executor.map(analyze_crypto_4hour, list_crypto)
      # # 1 hour
      # if (minutes_1_hour == 0) and second <= 1:
      #     print("\n",current_time.hour,":",current_time.minute , "For 1 hour we got :")
      #     with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
      #        executor.map(analyze_pair_coin_1hour, list_forex)
      #     with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
      #        executor.map(analyze_crypto_1hour, list_crypto)
      # # 30 min
      # if (minutes_30 == 0) and second <= 1:
      #     print("\n",current_time.hour,":",current_time.minute , "For 30 min we got :")
      #     with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
      #        executor.map(analyze_pair_coin_30min, list_forex)
      #     with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
      #        executor.map(analyze_crypto_30min, list_crypto)
      # 5 min
      if (minutes_5 == 0) and second <= 1:
          print("\n",current_time.hour,":",current_time.minute , "For 5 min we got :")
          for i in range(len(list_forex)):
            analyze_pair_coin_5min(list_forex[i])
          # for i in range(len(list_crypto)):
          #   analyze_crypto_5min(list_crypto[i])

if __name__ == "__main__" :
    main()


 16 : 40 For 5 min we got :
EURUSD
USDJPY
USDCHF
USDCAD
AUDUSD
GBPUSD
NZDUSD
EURNZD
EURCHF
EURGBP
EURCAD
EURAUD
EURJPY
GBPJPY
GBPNZD
GBPCHF
GBPCAD
GBPAUD
CADJPY
AUDJPY
CHFJPY
NZDJPY
AUDCHF
CADCHF
NZDCHF
AUDNZD
AUDCAD
NZDCAD

 16 : 45 For 5 min we got :
EURUSD
USDJPY
USDCHF
[ USDCHF buy ch per =  0.02 ]  USDCAD
AUDUSD
GBPUSD
[ GBPUSD sell ch per =  0.04 ]  NZDUSD
EURNZD
EURCHF
EURGBP
EURCAD
EURAUD
EURJPY
GBPJPY
GBPNZD
GBPCHF
[ GBPCHF buy ch per =  0.02 ]  GBPCAD
GBPAUD
CADJPY
AUDJPY
CHFJPY
NZDJPY
AUDCHF
CADCHF
NZDCHF
AUDNZD
AUDCAD
NZDCAD


KeyboardInterrupt: 